<a href="https://colab.research.google.com/github/Vadiman728/MountainCar_from_OpenAiGym_with_epsilon-greedy_strategy/blob/main/MountainCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Почему алгоритм с  𝛆 -жадной стратегией? а- Потому что для этой задачи не нужны сложные алгоритмы и она проста в реализации и б- эпсилон жадная стратегия позволяет агенту быстрее находить верное решение путем исследования и использования. В итоге мы получаем агента, который исследуя среду (состояние если угодно) находит верное решение. Но автор данного блокнота(я) столкнулся с очень интересной проблемой и применил интересное решение

Мы будем обучать агента, находящегося в среде MountainCar. В качестве агента выступает тележка, которая случайным образом расположена по оси x в диапазоне от -0,3 до -0,4. Максимальное вознаграждение находится в точке x = 0,5, но есть значительное изменение по оси y (подробности не важны).


Агент может выполнять три действия: 0 — тормозить, 1 — двигаться влево (в сторону отрицательных значений по оси x) и 2 — двигаться вправо (в сторону положительных значений по оси x). Это вся информация, которая нам нужна для анализа проблемы, с которой я столкнулся.

# Импорт библиотек


In [ ]:
!pip install --upgrade gym pyvirtualdisplay ipykernel > /dev/null 2>&1

In [ ]:
import torch
import gym
env = gym.make('MountainCar')

/usr/local/lib/python3.11/dist-packages/gym/envs/registration.py:563: UserWarning: WARN: Using the latest versioned environment `MountainCar-v0` instead of the unversioned environment `MountainCar`.
  logger.warn(


Версия среды импортируется сама

Получаем стартовое состояние

In [ ]:
state, _ = env.reset()

# Импорт функций

In [ ]:
import numpy as np
import torch
from collections import defaultdict
import math

В ходе тестирования возникла любопытная ситуация: агент не желал самостоятельно двигаться в направлении флага. Было странно, что каждые 50 эпизодов награда стабильно составляла -200 (окончание эпизода по времени), а не -199.(9), хотя и награда была принудительно установлена в формате float, среднее по 50 эпизодам стабильно было -200.0.


Пришлось потратить много времени, чтобы собрать список всех действий, преобразовать его во множество и проверить, применяется ли действие 2 (движение в сторону флага). Спойлер- нет.

Замена epsilon, динамичный epsilon, и уж тем более замена gamma (объяснены далее в коде) абсолютно никаким образом не влияли на поведение модели. Поэтому пришлось идти на принудительные меры, а точнее принудительное действие в нашем случае. Что если "показать" агенту, что за действие 2 она получит вознаграждение и это то действие, к которому ей надо стремиться? Тогда он поймет, что нужно действовать в это сторону. Опробуем!

In [ ]:
def run_episode(env, Q, epsilon, n_action, force_action=None):
    state, _ = tuple(env.reset())
    rewards = []
    actions = []
    states = []
    done = False
    truncated = False
    step_count = 0  # Счетчик шагов

    while not done and not truncated:
        state = tuple(state)

        # Принудительное действие
        if force_action is not None and step_count % 2 == 0:  # Каждые 10 шагов
            action = force_action
        else:
            probs = torch.ones(n_action) * epsilon / n_action
            if state in Q:
                best_action = torch.argmax(Q[state]).item()
                probs[best_action] += (1.0 - epsilon)
            else:
                best_action = torch.randint(0, n_action, [1]).item()

            action = torch.multinomial(probs, 1).item()

        actions.append(action)
        states.append(state)

        next_state, reward, done, truncated, info = env.step(action)

        # Изменение вознаграждения, чтобы поощрять действие 2
        if action == 2:  # Если агент катится вправо

            reward += 5  # Поощряем за это действие

        rewards.append(reward)

        # Отладочная информация, крашит браузер, запускать при достаточных ресурсах
        # print(f"State: {state}, Action: {action}, Reward: {reward}, Next State: {next_state}, Q-values: {Q[state]}")

        state = next_state
        step_count += 1

    return states, actions, rewards


Пишем стандартный алгоритм эпсилон- жадной стратегии

In [ ]:
# Реализация управления МК с ε-жадной единой стратегией
from collections import defaultdict
import math, random


def mc_control_epsilon_greedy(env, gamma, n_episode, epsilon):
    """
    Строит оптимальную ε-жадную стратегию методом управления МК с единой стратегией
    @param env: имя окружающей среды OpenAI Gym
    @param gamma: коэффициент обесценивания
    @param n_episode: количество эпизодов
    @param epsilon: компромисс между исследованием и использованием
    @return: оптимальные Q-функция и стратегия
    """
    n_action = env.action_space.n  # количество возможных действий
    G_sum = defaultdict(float)  # сумма возвратов для каждого состояния-действия
    N = defaultdict(int)  # количество посещений для каждого состояния-действия
    Q = defaultdict(lambda: torch.zeros(n_action))  # инициализация Q-функции
    temp_rewards = []  # буферный список

    for episode in range(n_episode):
        states_t, actions_t, rewards_t = run_episode(env, Q, epsilon, n_action)  # Выполнение эпизода
          # Обновление epsilon
        epsilon = max(min_epsilon, epsilon * epsilon_decay)

        return_t = 0  # инициализация возврата для текущего эпизода
        G = {}  # словарь для хранения возвратов для каждого состояния-действия
        m = math.fsum(rewards_t)  # находим общее вознаграждение для эпизода
        temp_rewards.append(m)  # сохраняем сумму в буферный список

        # Обратный проход по эпизоду для вычисления возвратов
        for state_t, action_t, reward_t in zip(states_t[::-1], actions_t[::-1], rewards_t[::-1]):
            return_t = gamma * return_t + reward_t  # вычисление возврата с учетом обесценивания
            G[(state_t, action_t)] = return_t  # сохранение возврата для текущего состояния-действия

        # Обновление Q-функции на основе возвратов

        if state_t[0] <= -0.:
          for state_action, return_t in G.items():
              state, action = state_action
              G_sum[state_action] += return_t
              N[state_action] += 1
              Q[state][action] = G_sum[state_action] / N[state_action]  # обновление Q-значения

        # Логирование результатов каждые 100 эпизодов
        if episode >= 1 and episode % 100 == 0:
            avg_reward = np.mean(temp_rewards[-100:])  # среднее вознаграждение за последние 100 эпизодов
            print(f'End of {episode + 1} episode')
            print(f'Episode: {episode + 1}, Epsilon: {epsilon}')
            print(f'Average Reward (last 100 episodes): {avg_reward}')

    # Формирование оптимальной стратегии на основе Q-функции
    print('Start of creating policy')
    policy = {}
    for state, actions in Q.items():
        policy[state] = torch.argmax(actions).item()  # выбор действия с максимальным Q-значением
    print('End of creating policy')

    return Q, policy  # возврат оптимальных Q-функции и стратегии


In [ ]:
# Задание параметров
gamma = 1.0  # коэффициент обесценивания
n_episode = 11000  # количество эпизодов
epsilon = 2.0  # Начальное значение epsilon
epsilon_decay = 0.999  # Коэффициент уменьшения epsilon
min_epsilon = 0.01  # Минимальное значение epsilon # компромисс между исследованием и использованием

In [ ]:
# Выполнение алгоритма управления МК с ε-жадной стратегией для нахождения оптимальных Q-функции и стратегии
optimal_Q, optimal_policy = mc_control_epsilon_greedy(env, gamma, n_episode, epsilon)

/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


End of 101 episode
Episode: 101, Epsilon: 1.8077747099331918
Average Reward (last 100 episodes): 132.7
End of 201 episode
Episode: 201, Epsilon: 1.6356603612983147
Average Reward (last 100 episodes): 134.1
End of 301 episode
Episode: 301, Epsilon: 1.4799326502478871
Average Reward (last 100 episodes): 136.35
End of 401 episode
Episode: 401, Epsilon: 1.3390314402014671
Average Reward (last 100 episodes): 127.8
End of 501 episode
Episode: 501, Epsilon: 1.2115451318326473
Average Reward (last 100 episodes): 131.95
End of 601 episode
Episode: 601, Epsilon: 1.096196521156022
Average Reward (last 100 episodes): 131.5
End of 701 episode
Episode: 701, Epsilon: 0.9918300040353356
Average Reward (last 100 episodes): 132.9
End of 801 episode
Episode: 801, Epsilon: 0.8973999989229295
Average Reward (last 100 episodes): 133.6
End of 901 episode
Episode: 901, Epsilon: 0.8119604718453174
Average Reward (last 100 episodes): 136.25
End of 1001 episode
Episode: 1001, Epsilon: 0.7346554586923851
Average 

Что же мы видим? уже с 2000го эпизода модель сама понимает (разница между вознаграждением) что нужно применять действие 2. Далее выведем лучшую стратегию, чтобы действительно увидеть, что действие 2 применяется не по нашей указке, а по добровольному решению агента

In [ ]:
# Вывод оптимальной стратегии
optimal_policy = {state: np.argmax(actions) for state, actions in optimal_Q.items()}
print("\nОптимальная стратегия (жадный выбор действий для каждого состояния):")
states_1= []
for state in optimal_policy:
    print(f"Состояние: {state}, Действие: {optimal_policy[state]}")
    states_1.append(optimal_policy[state])


Streaming output truncated to the last 5000 lines.
Состояние: (-0.5811866, 0.01266187), Действие: 0
Состояние: (-0.59384847, 0.013138888), Действие: 1
Состояние: (-0.60698736, 0.012519977), Действие: 0
Состояние: (-0.6195073, 0.011810547), Действие: 0
Состояние: (-0.6313179, 0.010016642), Действие: 0
Состояние: (-0.64133453, 0.010151869), Действие: 1
Состояние: (-0.6514864, 0.00921607), Действие: 0
Состояние: (-0.66070247, 0.009216541), Действие: 1
Состояние: (-0.669919, 0.007154044), Действие: 2
Состояние: (-0.67707306, 0.005043227), Действие: 2
Состояние: (-0.68211627, 0.0038986527), Действие: 1
Состояние: (-0.68601495, 0.0037281618), Действие: 0
Состояние: (-0.6897431, 0.0025330374), Действие: 1
Состояние: (-0.6922761, 0.0023212614), Действие: 0
Состояние: (-0.6945974, 9.4287214e-05), Действие: 2
Состояние: (-0.69469166, -0.002133303), Действие: 2
Состояние: (-0.6925584, -0.0023469303), Действие: 0
Состояние: (-0.6902115, -0.0025451386), Действие: 0
Состояние: (-0.6876663, -0.002726

Видно и подряд повторяющиеся действия 2 и через раз повторяющиеся, но своего мы добились- агент понял в какую сторону ему нужно двигаться для получения максимального вознаграждения

**А что дальше?**

Можно поиграться с параметрами вознаграждения за 2 дейстие. Можно увеличить цикличность 2го действия. Так же обязательно увеличить количество эпизодов, так как для такой задачи 11 000 эпизодов очень мало.